In [1]:
import pandas as pd
import QUANTAXIS as qtx
#from quantax.data_query_advance import *
#from quantax.data_query import *
from datetime import datetime, timedelta

In [2]:
TOP_STOCKS=500
CHECK_DAYS=30

In [3]:
def mtm_ind(data, col, N=12, M=6):
    mtm = (data[col] - qtx.REF(data[col], N)) / qtx.REF(data[col], N)
    mtmma = qtx.MA(mtm, M)
    return pd.DataFrame({'MTM': mtm, 'MTMMA': mtmma})

In [4]:
code_list = qtx.QA_fetch_stock_list()
finfo = qtx.QA_fetch_stock_info(list(code_list.code))
fcnd = qtx.QA_fetch_stock_day_adv(list(code_list.code),
                                  start=(datetime.today()-timedelta(days=CHECK_DAYS+30)).strftime('%Y-%m-%d'), 
                                  end=datetime.today().strftime('%Y-%m-%d')).to_qfq()


In [5]:
fbase = fcnd.data.join(finfo.loc[finfo.ipo_date != 0, ['ipo_date','industry']], how='inner').reset_index()
fbase['ipodays'] = fbase.apply(lambda x: (x.date - datetime.strptime(str(x.ipo_date), '%Y%m%d')).days, axis=1)
fbase = fbase.set_index(['date', 'code']) 
fbase.head()

open   high    low  close     volume        amount  adj  \
date       code                                                               
2021-01-04 000001  19.10  19.10  18.44  18.60  1554216.0  2.891682e+09  1.0   
           000002  28.40  28.40  27.52  27.78  1468441.0  4.079417e+09  1.0   
           000004  20.69  21.19  20.52  21.02    27398.0  5.726255e+07  1.0   
           000005   2.53   2.53   2.51   2.52    48963.0  1.231635e+07  1.0   
           000006   5.64   5.64   5.50   5.52    81326.0  4.502973e+07  1.0   

                   ipo_date  industry  ipodays  
date       code                                 
2021-01-04 000001  19910403         1    10869  
           000002  19910129        11    10933  
           000004  19910114        24    10948  
           000005  19901210        39    10983  
           000006  19920427        11    10479

In [6]:
mtm=fcnd.add_func(mtm_ind, col='close', N=20).loc[:, ['MTM','MTMMA']]
mtm.head()

MTM  MTMMA
date       code              
2021-01-04 000001  NaN    NaN
           000002  NaN    NaN
           000004  NaN    NaN
           000005  NaN    NaN
           000006  NaN    NaN

In [7]:
smtm = fbase.loc[fbase.ipodays > 180, :].join(mtm)
smtm.head()

open   high    low  close     volume        amount  adj  \
date       code                                                               
2021-01-04 000001  19.10  19.10  18.44  18.60  1554216.0  2.891682e+09  1.0   
           000002  28.40  28.40  27.52  27.78  1468441.0  4.079417e+09  1.0   
           000004  20.69  21.19  20.52  21.02    27398.0  5.726255e+07  1.0   
           000005   2.53   2.53   2.51   2.52    48963.0  1.231635e+07  1.0   
           000006   5.64   5.64   5.50   5.52    81326.0  4.502973e+07  1.0   

                   ipo_date  industry  ipodays  MTM  MTMMA  
date       code                                             
2021-01-04 000001  19910403         1    10869  NaN    NaN  
           000002  19910129        11    10933  NaN    NaN  
           000004  19910114        24    10948  NaN    NaN  
           000005  19901210        39    10983  NaN    NaN  
           000006  19920427        11    10479  NaN    NaN

In [8]:
smtm['rank'] = smtm.groupby('date')['MTM'].rank(method='dense', ascending=False)
smtm.loc[smtm['rank']<TOP_STOCKS,:].head(10)

open   high    low  close     volume        amount  adj  \
date       code                                                               
2021-02-01 000001  23.00  24.99  22.70  24.55  1475239.0  3.529557e+09  1.0   
           000026  12.63  13.04  12.20  12.60   109580.0  1.368616e+08  1.0   
           000050  16.17  17.14  16.07  16.96   403545.0  6.807325e+08  1.0   
           000100   8.70   9.19   8.60   9.03  4870139.0  4.341706e+09  1.0   
           000155  15.56  15.71  14.25  14.42  1474046.0  2.197157e+09  1.0   
           000157  12.46  13.66  12.26  13.58  2185961.0  2.869445e+09  1.0   
           000301  14.21  14.65  12.22  14.01   743802.0  1.032744e+09  1.0   
           000338  21.50  22.49  20.90  21.88  1707071.0  3.716208e+09  1.0   
           000403  41.42  41.50  40.08  40.47    17611.0  7.157565e+07  1.0   
           000408   8.80   9.35   8.73   8.77   221833.0  1.980523e+08  1.0   

                   ipo_date  industry  ipodays       MTM  MTMMA   rank  
date       code                                                         
2021-02-01 000001  19910403         1    10897  0.319892    NaN   58.0  
           000026  19930603        12    10105  0.065089    NaN  465.0  
           000050  19950315        35     9455  0.140551    NaN  240.0  
           000100  20040130        35     6212  0.257660    NaN   98.0  
           000155  20000926        16     7433  0.682614    NaN    5.0  
           000157  20001012        26     7417  0.328767    NaN   51.0  
           000301  20000529        27     7553  0.519523    NaN   11.0  
           000338  20070430         7     5026  0.365793    NaN   41.0  
           000403  19960628        34     8984  0.099728    NaN  340.0  
           000408  19960628        28     8984  0.108723    NaN  311.0

In [10]:
raw = pd.read_csv("c:\\Users\\mUSicX\\Documents\\股票相关\\20201030SW.csv")
raw['c'] = raw.code.apply(lambda x: x[:6])
raw['mkt'] = raw.code.apply(lambda x: 0 if x.endswith('SZ') else 1)
raw['ind_cat1'] = raw.ind.apply(lambda x: x.split('--')[0])
raw['ind_cat2'] = raw.ind.apply(lambda x: x.split('--')[1].replace('Ⅱ', ''))
raw['ind_cat3'] = raw.ind.apply(lambda x: x.split('--')[2].replace('Ⅲ', ''))

raw['ind_cat'] = raw.apply(lambda x: x.ind_cat1+'-'+x.ind_cat2, axis=1)
raw[:10]

,code,ind,c,mkt,ind_cat1,ind_cat2,ind_cat3,ind_cat
0,000009.SZ,综合--综合Ⅱ--综合Ⅲ,000009,0,综合,综合,综合,综合-综合
1,000409.SZ,综合--综合Ⅱ--综合Ⅲ,000409,0,综合,综合,综合,综合-综合
2,000632.SZ,综合--综合Ⅱ--综合Ⅲ,000632,0,综合,综合,综合,综合-综合
3,000638.SZ,综合--综合Ⅱ--综合Ⅲ,000638,0,综合,综合,综合,综合-综合
4,000652.SZ,综合--综合Ⅱ--综合Ⅲ,000652,0,综合,综合,综合,综合-综合
5,000839.SZ,综合--综合Ⅱ--综合Ⅲ,000839,0,综合,综合,综合,综合-综合
6,002967.SZ,综合--综合Ⅱ--综合Ⅲ,002967,0,综合,综合,综合,综合-综合
7,003008.SZ,综合--综合Ⅱ--综合Ⅲ,003008,0,综合,综合,综合,综合-综合
8,300012.SZ,综合--综合Ⅱ--综合Ⅲ,300012,0,综合,综合,综合,综合-综合
9,300688.SZ,综合--综合Ⅱ--综合Ⅲ,300688,0,综合,综合,综合,综合-综合


In [11]:
industry = raw.loc[:, ['c', 'ind_cat']].set_index('c').to_dict()
indict = industry['ind_cat']

In [12]:
smtm = smtm.reset_index()
smtm['cat'] = smtm.apply(lambda x: indict[str(x.code)], axis=1)
smtm = smtm.loc[:, ['date', 'code', 'cat', 'rank']]
smtm.head(10)

,date,code,cat,rank
0,2021-01-04,000001,银行-银行,NaN
1,2021-01-04,000002,房地产-房地产开发,NaN
2,2021-01-04,000004,医药生物-化学制药,NaN
3,2021-01-04,000005,公用事业-环保工程及服务,NaN
4,2021-01-04,000006,房地产-房地产开发,NaN
5,2021-01-04,000007,商业贸易-商业物业经营,NaN
6,2021-01-04,000008,机械设备-运输设备,NaN
7,2021-01-04,000009,综合-综合,NaN
8,2021-01-04,000010,建筑装饰-园林工程,NaN
9,2021-01-04,000011,房地产-房地产开发,NaN


In [13]:
tops = smtm.loc[smtm['rank']<TOP_STOCKS,:].groupby(['date', 'cat'])['code'].count()
bases = smtm.groupby(['date', 'cat'])['code'].count()
rps = pd.merge(tops.reset_index(), bases.reset_index(), on=['date', 'cat'])
rps.rename(columns={'code_x':'high', 'code_y':'full'}, inplace=True)
rps.head(10)

,date,cat,high,full
0,2021-02-01,交通运输-物流,7,40
1,2021-02-01,交通运输-航空运输,2,8
2,2021-02-01,休闲服务-其他休闲服务,1,3
3,2021-02-01,传媒-互联网传媒,10,58
4,2021-02-01,传媒-文化传媒,10,75
5,2021-02-01,传媒-营销传播,5,34
6,2021-02-01,公用事业-燃气,2,24
7,2021-02-01,公用事业-环保工程及服务,5,55
8,2021-02-01,公用事业-电力,5,75
9,2021-02-01,农林牧渔-农产品加工,3,17


In [14]:
rps['score'] = rps.high / rps.full * rps.high
rps['rank'] = rps.groupby('date')['score'].rank(ascending=False, method='dense', na_option='bottom')
rps['cap'] = rps.apply(lambda r: '{}/{}'.format(r.high, r.full), axis=1)
rps.head(10)

,date,cat,high,full,score,rank,cap
0,2021-02-01,交通运输-物流,7,40,1.225000,25.0,7/40
1,2021-02-01,交通运输-航空运输,2,8,0.500000,38.0,2/8
2,2021-02-01,休闲服务-其他休闲服务,1,3,0.333333,45.0,1/3
3,2021-02-01,传媒-互联网传媒,10,58,1.724138,16.0,10/58
4,2021-02-01,传媒-文化传媒,10,75,1.333333,23.0,10/75
5,2021-02-01,传媒-营销传播,5,34,0.735294,33.0,5/34
6,2021-02-01,公用事业-燃气,2,24,0.166667,53.0,2/24
7,2021-02-01,公用事业-环保工程及服务,5,55,0.454545,41.0,5/55
8,2021-02-01,公用事业-电力,5,75,0.333333,45.0,5/75
9,2021-02-01,农林牧渔-农产品加工,3,17,0.529412,37.0,3/17


In [15]:
rps = rps.loc[rps.date > (datetime.today()-timedelta(days=CHECK_DAYS)).strftime('%Y-%m-%d'), :]

In [16]:
ind_pivot = rps.pivot(index='cat', columns='date', values=['score','rank', 'cap'])
ind_pivot = ind_pivot.reindex(sorted(ind_pivot.columns, reverse=True), axis=1)
ind_pivot.fillna(0, inplace=True)
ind_pivot.sort_values(by=ind_pivot.columns[0], ascending=False, inplace=True)
top_ind = ind_pivot.loc[(ind_pivot.score > 1).any(axis=1),:]
top_ind

score                                              \
date         2021-03-01 2021-02-26 2021-02-25 2021-02-24 2021-02-23   
cat                                                                   
化工-化学原料       11.428571  13.444444   9.000000   9.000000   8.027778   
化工-化学制品        9.240175  10.530702   7.017544  13.267544  14.250000   
有色金属-工业金属      6.942308   8.480769   7.692308  13.254902  14.294118   
休闲服务-景点        5.818182   5.818182   5.818182   3.272727   4.454545   
公用事业-环保工程及服务   5.070175   4.982759   1.396552   1.103448   0.620690   
机械设备-专用设备      4.712042   2.534031   2.308901   2.321053   3.031579   
休闲服务-旅游综合      4.083333   4.083333   5.333333   5.333333   2.083333   
化工-化学纤维        4.000000   2.560000   4.000000   4.000000   6.760000   
轻工制造-造纸        3.200000   1.800000   1.800000   2.450000   1.800000   
传媒-互联网传媒       2.913793   2.482759   1.396552   2.913793   1.103448   
有色金属-稀有金属      2.666667   2.560000   1.960000   6.760000   7.840000   
农林牧渔-饲料        2.400000   2.400000   2.400000   2.400000   2.400000   
机械设备-通用机械      2.368852   2.959016   2.655738   1.859504   1.619835   
休闲服务-其他休闲服务    2.250000   2.250000   1.000000   2.250000   0.250000   
交通运输-航空运输      2.000000   2.000000   2.000000   2.000000   2.000000   
化工-石油化工        1.894737   0.842105   0.842105   0.473684   0.842105   
纺织服装-服装家纺      1.886792   1.528302   1.528302   0.924528   0.471698   
家用电器-视听器材      1.454545   0.818182   0.818182   0.363636   0.363636   
钢铁-钢铁          1.400000   1.058824   1.441176   0.470588   1.441176   
商业贸易-专业零售      1.333333   1.454545   0.818182   0.333333   0.083333   
建筑装饰-装修装饰      1.333333   0.925926   0.925926   0.333333   0.148148   
计算机-计算机应用      1.299492   0.614213   0.324873   0.507614   0.614213   
非银金融-多元金融      1.285714   0.571429   0.310345   0.310345   0.310345   
房地产-房地产开发      1.263158   2.245614   2.188034   1.034188   0.551724   
轻工制造-家用轻工      1.157143   1.428571   1.728571   0.700000   1.157143   
建筑材料-玻璃制造      1.125000   0.500000   1.125000   0.500000   0.125000   
传媒-文化传媒        1.080000   1.333333   1.333333   1.080000   1.080000   
医药生物-化学制药      0.925926   0.925926   2.348624   2.651376   3.311927   
电子-电子制造        0.914286   0.514286   0.514286   0.357143   0.514286   
采掘-其他采掘        0.900000   0.400000   0.900000   1.600000   2.500000   
建筑材料-水泥制造      0.888889   1.388889   2.000000   0.222222   0.222222   
化工-橡胶          0.842105   0.473684   0.842105   0.842105   1.315789   
有色金属-黄金        0.818182   0.818182   1.454545   2.083333   3.000000   
建筑材料-其他建材      0.818182   0.204545   1.113636   0.568182   0.204545   
休闲服务-酒店        0.800000   1.800000   2.666667   2.666667   2.666667   
综合-综合          0.781250   1.531250   0.500000   0.281250   0.500000   
汽车-汽车零部件       0.729927   0.729927   0.883212   0.729927   1.051095   
采掘-煤炭开采        0.657895   0.657895   1.289474   1.684211   2.131579   
轻工制造-包装印刷      0.641026   0.923077   0.410256   0.657895   0.421053   
采掘-采掘服务        0.600000   1.666667   1.066667   0.600000   1.666667   
建筑装饰-专业工程      0.533333   0.833333   0.833333   0.533333   0.533333   
医药生物-医疗器械      0.486486   0.216216   0.864865   2.315068   2.315068   
电子-光学光电子       0.467532   1.282051   1.282051   1.038462   0.820513   
电气设备-电源设备      0.308642   0.200000   0.444444   1.012500   1.512500   
交通运输-物流        0.219512   0.609756   0.390244   0.219512   0.609756   
建筑装饰-房屋建设      0.200000   0.450000   1.250000   0.200000   0.200000   
汽车-汽车整车        0.181818   0.181818   0.181818   0.181818   0.181818   
家用电器-白色家电      0.180000   0.500000   0.320000   0.500000   0.180000   
医药生物-医疗服务      0.166667   0.166667   0.666667   1.041667   2.041667   
银行-银行          0.111111   0.694444   1.361111   0.250000   0.250000   
农林牧渔-动物保健      0.100000   0.400000   1.600000   0.900000   1.600000   
医药生物-生物制品      0.090909   0.363636   0.568182   0.568182   1.454545   
电气设备-电气自动化设备   0.086957   0.088889   0.088889   0.088889   0.088889   
电气设备-电机        0.062500   0.000000   0.250

In [18]:
top_ind.to_excel('c:\\Users\\mUSicX\\Desktop\\block_rps_scores{}.xls'.format(TOP_STOCKS))

In [ ]:
mtest = mtm.reset_index()
mtest.loc[mtest.code=='300758',:]